In [5]:
#!/usr/bin/env python
# coding: utf-8

# # SST LR

# In[1]:


import numpy as np


def concatenate(X):
    return np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)


def error(X, y, model):
    return model.predict_proba(X)[:, 1] - y


def prediction(X, model):
    return model.predict_proba(X)[:, 1]


def gradient_(X, y, model, C):
    F = concatenate(X)
    #return F * error(X, y, model)[:, None] + C * w / X.shape[0]
    return F * error(X, y, model)[:, None]


def hessian_(X, model, C):
    probs = prediction(X, model)
    F = concatenate(X)
    H = F.T @ np.diag(probs * (1 - probs)) @ F / F.shape[0] + C * np.eye(
        F.shape[1]) / X.shape[0]
    return H


def inverse_hessian(H):
    return np.linalg.inv(H)


# In[2]:


def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
    
    X_r = X["train"][top_k_index]
    y_r = y["train"][top_k_index]
    X_l = np.delete(X["train"], top_k_index, axis=0)
    y_l = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_l, y_l, prediction, X_r, y_r


# In[3]:


def new_train(k, dev_index, scores):
    X_k, y_k, prediction, x_r, y_r = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    return new


# In[4]:


def approximate_k(test_idx):
    old = pred[test_idx].item()
    for k in range(1, np.sum(y["train"])):
        X_l, y_l, pred_change_if, X_r, y_r = Remove(k, delta_pred, test_idx)
        #_, flip,_ = new_train(k, test_idx, delta_pred)
        
        if old > 0.5 and pred_change_if + old < 0.5:
            return k
        elif old < 0.5 and pred_change_if + old > 0.5:
            return k
        
    return None


# In[60]:


def recursive_NT(test_idx, old, X_0, y_0, model, l2, I=100, D=1):
    eps = 1 / X["train"].shape[0]
    X_l = X_0
    y_l = y_0
    X_r = X_0
    y_r = y_0
    # Compute IP
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)
    new_hessain = hessian_(X_l, model, l2)
    new_inv = inverse_hessian(new_hessain)
    new_grad_train = gradient_(X_r, y_r, model, l2)
    delta_k = -eps * new_inv @ new_grad_train.T
    grad_f = F_dev[test_idx] * (old * (1 - old))
    delta_pred = grad_f @ delta_k

    K_new = y_0.shape[0]
    predicted_change_new = None
    ite = 0
    diff = K_new - 0
    removed_order = []
    while (ite < I and diff > D and K_new != 1):
        ite += 1

        if old > 0.5:
            sorted_index = np.flip(delta_pred.argsort())
        else:
            sorted_index = delta_pred.argsort()

        for k in range(1, y_r.shape[0]):
            top_k_index = sorted_index[:k]
            predicted_change = -np.sum(delta_pred[top_k_index])
            """
            print("K", k)
            print("predicted_change", predicted_change)
            print("old", np.round(old))
            print("K", k)
            print("removed shape", X_r.shape)
            print("left shape", X_l.shape)
            print("delta_pred shape", delta_pred.shape)
            print("predicted_change", predicted_change)
            print("old", np.round(old), "new", np.round(old + predicted_change))
            print()
            """

            if np.round(old) != np.round(old + predicted_change):
                # print("K", k)

                diff = K_new - k
                K_new = k
                predicted_change_new = predicted_change

                index_whole = []
                for idx_r in top_k_index:
                    point = X_r[idx_r]
                    idx_0 = X_dist[str(point.tolist())]
                    index_whole.append(idx_0)

                X_r = X_r[top_k_index]
                y_r = y_r[top_k_index]

                X_l = np.delete(X_0, index_whole, axis=0)
                y_l = np.delete(y_0, index_whole, axis=0)
                # print("removed shape", X_r.shape)
                # print("left shape", X_l.shape)

                # new hessian for the left points
                new_hessain = hessian_(X_l, model, C=l2)
                new_inv = inverse_hessian(new_hessain)
                # new gradient for the removed points
                new_grad_train = gradient_(X_r, y_r, model, C=l2)

                delta_k = -eps * new_inv @ new_grad_train.T
                grad_f = F_dev[test_idx] * (old * (1 - old))
                delta_pred = grad_f @ delta_k
                break

            if k == y_r.shape[0] - 1:
                if K_new == y_0.shape[0] and diff == y_0.shape[0]:
                    return None, predicted_change_new, ite, None, []
                
                return K_new, predicted_change_new, ite, diff, index_whole

    return K_new, predicted_change_new, ite, diff, index_whole


# In[6]:

In [7]:
import numpy as np
from tqdm import tqdm
import spacy
import urllib.request, json
from sklearn.linear_model import LogisticRegression

def SST2():
    sst_dataset = {}
    for split in ["train", "dev", "test"]:
        URL = f"https://raw.githubusercontent.com/successar/instance_attributions_NLP/master/Datasets/SST/data/{split}.jsonl"
        with urllib.request.urlopen(URL) as url:
            data = url.read().decode()
            data = [json.loads(line) for line in data.strip().split("\n")]
            sst_dataset[split] = data

    nlp = spacy.load('en_core_web_md')
    #nlp = spacy.load('en_core_web_sm')
    # We use spacy to get feature vectors for our input text.

    X, y = {}, {}
    for split in ["train", "dev"]:
        X[split] = np.array([nlp(example["document"]).vector for example in tqdm(sst_dataset[split])])
        y[split] = np.array([example["label"] for example in sst_dataset[split]])

    return X, y

In [9]:
X, y = SST2()


 41%|███████████████▍                      | 2818/6920 [00:21<00:25, 158.20it/s]


 87%|█████████████████████████████████     | 6019/6920 [00:41<00:05, 155.73it/s]


100%|██████████████████████████████████████| 6920/6920 [00:47<00:00, 145.83it/s]

100%|████████████████████████████████████████| 872/872 [00:06<00:00, 139.40it/s]


In [10]:
l2 = 1
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

# compute IP for new train
from sklearn.preprocessing import normalize
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 1 * np.eye(F_train.shape[1]) / X["train"].shape[0]
H_inv = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * H_inv @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k

In [11]:
"""
X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i
    
NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print("order", len(order))
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_SST_LR_I100_D1_.npy", NT_app_k)
np.save("new_predictions_SST_LR_I100_D1_.npy", new_predictions)
np.save("iterations_SST_LR_I100_D1_.npy", iterations)
np.save("diffs_SST_LR_I100_D1_.npy", diffs)
np.save("order_SST_LR_I100_D1_.npy", order_lists)

test_idx 0
old 0.9866086180079268
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 1
old 0.3009748953355743
K_nt, pred_change_nt, ite, diff
45 0.201756171912998 3 8
new 0.5334349848817815
order 45

test_idx 2
old 0.9990631406484662
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 3
old 0.9699749786556093
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 4
old 0.9996985847106401
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 5
old 0.9376713051932866
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 6
old 0.9723592196755176
K_nt, pred_change_nt, ite, diff
None None 1 None
new None
order 0

test_idx 7
old 0.3251752573164711


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
43 0.17663352814489908 3 6
new 0.5204760357574684
order 43

test_idx 8
old 0.83933264256975


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
168 -0.3404584296917055 3 45
new 0.2747123717690886
order 168

test_idx 9
old 0.1762500981801558


...

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
13 -0.08414643334755971 3 2
new 0.49433188257462957
order 13

test_idx 866
old 0.20860223733825017
K_nt, pred_change_nt, ite, diff
78 0.2920531958998262 3 25
new 0.6144740175215819
order 78

test_idx 867
old 0.5330097261570403
K_nt, pred_change_nt, ite, diff
4 -0.03630861183788763 2 6916
new 0.49259301760318164
order 4

test_idx 868
old 0.5366908337223938
K_nt, pred_change_nt, ite, diff
8 -0.038480738322188 2 1
new 0.49792421361898204
order 8

test_idx 869
old 0.2121005712822696
K_nt, pred_change_nt, ite, diff
162 0.28829832548699813 3 56
new 0.6161076347496471
order 162

test_idx 870
old 0.08367276020147513


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
542 0.41660873418266814 3 426
new 0.9768573714897795
order 542

test_idx 871
old 0.23984703527489903
K_nt, pred_change_nt, ite, diff
109 0.2605558341950147 3 27
new 0.5799613601679013
order 109



/opt/anaconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [14]:

X = {}
y = {}
path = ""/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_SST/"
X["train"] = np.load("train_feature_SST.npy")
y["train"] = np.load("train_label_SST.npy").squeeze()
X["dev"] = np.load("test_feature_SST.npy")
y["dev"] = np.load("test_label_SST.npy").squeeze()


# In[14]:


def new_train(k, dev_index, scores):
    X_k, y_k, prediction, x_r, y_r = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    return new


# In[15]:


l2 = 10
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]

#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_H @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape


# In[18]:


X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i

NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_SST_bertT_I100_D1.npy", NT_app_k)
np.save("new_predictions_SST_bertT_I100_D1.npy", new_predictions)
np.save("iterations_SST_bertT_I100_D1.npy", iterations)
np.save("diffs_SST_bertT_I100_D1.npy", diffs)
np.save("order_SST_bertT_I100_D1.npy", order_lists)

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


test_idx 0
old 0.9638060040189218
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 1
old 0.33397931376862416


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
77 0.16614593351978846 3 15
new 0.5195656333308112

test_idx 2
old 0.9671392354824311
K_nt, pred_change_nt, ite, diff
None None 1 None
new None

test_idx 3
old 0.8238159816255983


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
781 -0.3804500218955019 4 31
new 0.12813363713506307

test_idx 256
old 0.21599249412836277


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
173 0.2845796218075449 3 39
new 0.6183006459983

test_idx 257
old 0.542360189668293


KeyboardInterrupt: 